Project: **Data Cleaning for the CDC Chronic Disease Indicators (CDI): Diabetes Dataset**

This code was written by Zainab Haider

Jupyter Notebook/Python

Date: 08/24/2020

*Library:*

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

db = pd.read_csv(r'...\CDI Diabetes.csv')

In [3]:
db.head()

,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,Response,DataValueUnit,DataValueType,...,LocationID,TopicID,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1,StratificationCategoryID2,StratificationID2,StratificationCategoryID3,StratificationID3
0,2010,2010,VA,Virginia,NVSS,Diabetes,Mortality with diabetic ketoacidosis reported ...,NaN,"cases per 100,000",Age-adjusted Rate,...,51,DIA,DIA1_2,AGEADJRATE,RACE,API,NaN,NaN,NaN,NaN
1,2010,2010,NY,New York,NVSS,Diabetes,Mortality due to diabetes reported as any list...,NaN,NaN,Number,...,36,DIA,DIA1_1,NMBR,RACE,WHT,NaN,NaN,NaN,NaN
2,2010,2010,OK,Oklahoma,NVSS,Diabetes,Mortality with diabetic ketoacidosis reported ...,NaN,"cases per 100,000",Crude Rate,...,40,DIA,DIA1_2,CRDRATE,GENDER,GENF,NaN,NaN,NaN,NaN
3,2010,2010,NH,New Hampshire,NVSS,Diabetes,Mortality due to diabetes reported as any list...,NaN,"cases per 100,000",Crude Rate,...,33,DIA,DIA1_1,CRDRATE,OVERALL,OVR,NaN,NaN,NaN,NaN
4,2010,2010,AK,Alaska,NVSS,Diabetes,Mortality with diabetic ketoacidosis reported ...,NaN,"cases per 100,000",Crude Rate,...,2,DIA,DIA1_2,CRDRATE,RACE,WHT,NaN,NaN,NaN,NaN


In [4]:
db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110251 entries, 0 to 110250
Data columns (total 34 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   YearStart                  110251 non-null  int64  
 1   YearEnd                    110251 non-null  int64  
 2   LocationAbbr               110251 non-null  object 
 3   LocationDesc               110251 non-null  object 
 4   DataSource                 110251 non-null  object 
 5   Topic                      110251 non-null  object 
 6   Question                   110251 non-null  object 
 7   Response                   0 non-null       float64
 8   DataValueUnit              102310 non-null  object 
 9   DataValueType              110251 non-null  object 
 10  DataValue                  60259 non-null   float64
 11  DataValueAlt               60259 non-null   object 
 12  DataValueFootnoteSymbol    51497 non-null   object 
 13  DatavalueFootnote          51

In [5]:
print("Total Number of Entries: " + str(db.shape[0]))
print("Total Number of Columns: " + str(db.shape[1]))

Total Number of Entries: 110251
Total Number of Columns: 34


In [6]:
missing = (db.isnull().sum())
print("Columns with Missing Data: ")
print(missing[missing > 0])

Columns with Missing Data: 
Response                     110251
DataValueUnit                  7941
DataValue                     49992
DataValueAlt                  49992
DataValueFootnoteSymbol       58754
DatavalueFootnote             58754
LowConfidenceLimit            55234
HighConfidenceLimit           55234
StratificationCategory2      110251
Stratification2              110251
StratificationCategory3      110251
Stratification3              110251
GeoLocation                     634
ResponseID                   110251
StratificationCategoryID2    110251
StratificationID2            110251
StratificationCategoryID3    110251
StratificationID3            110251
dtype: int64


If a column is missing all data, then I will delete it.

In [7]:
m = missing.loc[missing == db.shape[0]]
allmissing = m.index
for k in range(len(allmissing)):
    db = db.drop([allmissing[k]], axis=1)
    
print("Total Number of Columns: " + str(db.shape[1]))
print("Columns with Missing Data: ")
updatedmissing = (db.isnull().sum())
print(updatedmissing[missing > 0])

Total Number of Columns: 24
Columns with Missing Data: 
DataValueUnit               7941
DataValue                  49992
DataValueAlt               49992
DataValueFootnoteSymbol    58754
DatavalueFootnote          58754
LowConfidenceLimit         55234
HighConfidenceLimit        55234
GeoLocation                  634
dtype: int64


Next, I will be deleting columns that contains reduntant information: the Topic and TopicID column. These columns were relevant in the larger CDI dataset to ensure that the data was being attributed to the appropriate disease. However, this dataset is exclusively looking at diabetes data, so we do not need these two columns.

In [8]:
db = db.drop('Topic', axis=1)
db = db.drop('TopicID', axis=1)

One thing to note in this dataset is that there are multiple variables being used to express the same information in different way. For example: StratificationCategory1 and StratificationCategoryID1. Both convey the exact same information, so I will delete the "ID" type of columns

There are four columns for Location that all convey the same information. I will be deleting the Geolocation and LocationID columns, because that information is hard to interpret on their own and do not add value to the dataset.

In [9]:
print("Number of unique values in LocationAbbr variable: " + str(len(db.LocationAbbr.unique())))
print("Number of unique values in LocationDesc variable: " + str(len(db.LocationDesc.unique())))
print("Number of unique values in GeoLocation variable: " + str(len(db.GeoLocation.unique())))
print("Number of unique values in LocationID variable: " + str(len(db.LocationID.unique())))

Number of unique values in LocationAbbr variable: 55
Number of unique values in LocationDesc variable: 55
Number of unique values in GeoLocation variable: 55
Number of unique values in LocationID variable: 55


In [10]:
db = db.drop('GeoLocation', axis=1)
db = db.drop('LocationID', axis=1)

I will be dropping the Columns: DataValueTypeID, StratificationCategoryID1, StratificationID1, because they are abbrevations for the columns DataValueType, StratificationCategory1, Stratification1 , respectively, and do not contain new/valuable information.

In [11]:
db = db.drop('DataValueTypeID', axis=1)
db = db.drop('StratificationCategoryID1', axis=1)
db = db.drop('StratificationID1', axis=1)
db = db.drop('DataValueFootnoteSymbol', axis=1)
db = db.drop('DataValueAlt', axis=1)
db = db.drop('YearEnd', axis=1)

In [12]:
db.Question.unique()

array(['Mortality with diabetic ketoacidosis reported as any listed cause of death',
       'Mortality due to diabetes reported as any listed cause of death',
       'Hospitalization with diabetes as a listed diagnosis',
       'Prevalence of gestational diabetes',
       'Prevalence of pre-pregnancy diabetes',
       'Amputation of a lower extremity attributable to diabetes',
       'Foot examination among adults aged >= 18 years with diagnosed diabetes',
       'Glycosylated hemoglobin measurement among adults aged >= 18 years with diagnosed diabetes',
       'Pneumococcal vaccination among noninstitutionalized adults aged >= 65 years with diagnosed diabetes',
       'Dilated eye examination among adults aged >= 18 years with diagnosed diabetes',
       'Pneumococcal vaccination among noninstitutionalized adults aged 18-64 years with diagnosed diabetes',
       'Prevalence of diagnosed diabetes among adults aged >= 18 years',
       'Diabetes prevalence among women aged 18-44 years',

From 2010-2017, each state (and District of Columbia, Virgin Islands, Guam, Puerto Rico) in this dataset has provided data for 20 questions (some with multiple types of answers). These questions are answered based on gender, race/ethnicity, and on an overall basis. Thus, it is more efficient to seperate this data into three groups: overall data, data based on gender, and data based on race/ethnicity.


In [13]:
print("Number of Questions: " + str(db.Question.nunique()))
print("Number of Stratification Categories: " + str(db.StratificationCategory1.nunique()))

Number of Questions: 20
Number of Stratification Categories: 3


In [14]:
overall_db = db.loc[db.StratificationCategory1 == 'Overall']
gender_db = db.loc[db.StratificationCategory1 == 'Gender']
race_db = db.loc[db.StratificationCategory1 == 'Race/Ethnicity']

Next, I will delete the StratificationCategory1 variable from these new datasets, because this information is now redundant.

In [15]:
overall_db = overall_db.drop('StratificationCategory1', axis=1)
overall_db = overall_db.drop('Stratification1', axis=1)
gender_db = gender_db.drop('StratificationCategory1', axis=1)
race_db = race_db.drop('StratificationCategory1', axis=1)

The next think to look at is how much missing data we have. I am going to start with **Mortality due to diabetes reported as any listed cause of death**. For this, it is actually easier to look at it through the specific stratifications. Makes the data easier to follow.
Also, there is no data for multiracial, non-hispanic and other, non-hispanic ethnicities so we are not going to be considering those two ethnicities for mortality due to diabetes

In [16]:
print("Missing values for the Question: Mortality due to diabetes reported as any listed cause of death - Age-adjusted Rate")
print("Overall:")
years = [2010,2011,2012,2013,2014,2015,2016,2017]
mortality = overall_db.loc[overall_db.Question == 'Mortality due to diabetes reported as any listed cause of death']
for k in range(len(years)):
    m = mortality.loc[(mortality.DataValueType == 'Age-adjusted Rate') & (mortality.YearStart == years[k])]
    missing = (m['DataValue'].isnull().sum())
    if missing != 0:
        print(str(years[k]) + ": " + str(missing))

Missing values for the Question: Mortality due to diabetes reported as any listed cause of death - Age-adjusted Rate
Overall:


In [17]:
print("Missing values for the Question: Mortality due to diabetes reported as any listed cause of death - Age-adjusted Rate")
years = [2010,2011,2012,2013,2014,2015,2016,2017]
gender = ['Male','Female']

for g in range(len(gender)):
    print(gender[g] + ":")
    mortality = gender_db.loc[gender_db.Question == 'Mortality due to diabetes reported as any listed cause of death']
    for k in range(len(years)):
        m = mortality.loc[(mortality.DataValueType == 'Age-adjusted Rate') & (mortality.YearStart == years[k]) & (mortality.Stratification1 == gender[g])]
        missing = (m['DataValue'].isnull().sum())
        if missing != 0:
            print(str(years[k]) + ": " + str(missing))

Missing values for the Question: Mortality due to diabetes reported as any listed cause of death - Age-adjusted Rate
Male:
Female:


In [18]:
print("Missing values for the Question: Mortality due to diabetes reported as any listed cause of death - Age-adjusted Rate")
years = [2010,2011,2012,2013,2014,2015,2016,2017]
ethnicity = ['Asian or Pacific Islander', 'White, non-Hispanic', 'Hispanic',
       'Black, non-Hispanic', 'American Indian or Alaska Native']

for r in range(len(ethnicity)):
    print(ethnicity[r] + ":")
    mortality = race_db.loc[race_db.Question == 'Mortality due to diabetes reported as any listed cause of death']
    for k in range(len(years)):
        m = mortality.loc[(mortality.DataValueType == 'Age-adjusted Rate') & (mortality.YearStart == years[k]) & (mortality.Stratification1 == ethnicity[r])]
        missing = (m['DataValue'].isnull().sum())
        if missing != 0:
            print(str(years[k]) + ": " + str(missing))

Missing values for the Question: Mortality due to diabetes reported as any listed cause of death - Age-adjusted Rate
Asian or Pacific Islander:
2010: 23
2011: 25
2012: 23
2013: 23
2014: 21
2015: 20
2016: 17
2017: 18
White, non-Hispanic:
Hispanic:
2010: 13
2011: 13
2012: 12
2013: 14
2014: 11
2015: 12
2016: 8
2017: 11
Black, non-Hispanic:
2010: 11
2011: 11
2012: 11
2013: 11
2014: 11
2015: 11
2016: 11
2017: 11
American Indian or Alaska Native:
2010: 32
2011: 30
2012: 29
2013: 28
2014: 27
2015: 29
2016: 28
2017: 28


Next: we are going to look at the **Prevalence of diagnosed diabetes among adults aged >= 18 years**

There is not much data missing from overall and gender for prevalence of diabetes diagnosis. But, there is some missing data for ethnicity. The only ethnicities with available data are: Black, Multiracial, White, Other, Hispanic.

In [19]:
g = gender_db.loc[gender_db.Question == 'Prevalence of diagnosed diabetes among adults aged >= 18 years']
r = race_db.loc[race_db.Question == 'Prevalence of diagnosed diabetes among adults aged >= 18 years']
print(g.Stratification1.unique())
print(r.Stratification1.unique())

['Female' 'Male']
['Black, non-Hispanic' 'Multiracial, non-Hispanic' 'White, non-Hispanic'
 'Other, non-Hispanic' 'Hispanic']


In [20]:
print("Missing values for the Question: Prevalence of diabetes - Age-adjusted Prevalence")
print("Overall:")
years = [2011,2012,2013,2014,2015,2016,2017,2018]
prev = overall_db.loc[overall_db.Question == 'Prevalence of diagnosed diabetes among adults aged >= 18 years']
for k in range(len(years)):
    p = prev.loc[(prev.DataValueType == 'Age-adjusted Prevalence') & (prev.YearStart == years[k])]
    missing = (p['DataValue'].isnull().sum())
    if missing != 0:
        print(str(years[k]) + ": " + str(missing))

Missing values for the Question: Prevalence of diabetes - Age-adjusted Prevalence
Overall:
2011: 1
2012: 1
2013: 1
2014: 1
2015: 1
2017: 1
2018: 1


In [21]:
print("Missing values for the Question: Prevalence of diabetes - Age-adjusted Prevalence")
years = [2011,2012,2013,2014,2015,2016,2017,2018]
gender = ['Male','Female']

for g in range(len(gender)):
    print(gender[g] + ":")
    prev = gender_db.loc[gender_db.Question == 'Prevalence of diagnosed diabetes among adults aged >= 18 years']
    for k in range(len(years)):
        p = prev.loc[(prev.DataValueType == 'Age-adjusted Prevalence') & (prev.YearStart == years[k]) & (prev.Stratification1 == gender[g])]
        missing = (p['DataValue'].isnull().sum())
        if missing != 0:
            print(str(years[k]) + ": " + str(missing))

Missing values for the Question: Prevalence of diabetes - Age-adjusted Prevalence
Male:
2011: 1
2012: 1
2013: 1
2014: 1
2015: 1
2017: 1
2018: 1
Female:
2011: 1
2012: 1
2013: 1
2014: 1
2015: 1
2017: 1
2018: 1


In [22]:
print("Missing values for the Question: Prevalence of diabetes - Age-adjusted Prevalence")
ethnicity = ['Black, non-Hispanic', 'Multiracial, non-Hispanic', 'White, non-Hispanic',
 'Other, non-Hispanic', 'Hispanic']

for r in range(len(ethnicity)):
    print(ethnicity[r] + ":")
    prev = race_db.loc[race_db.Question == 'Prevalence of diagnosed diabetes among adults aged >= 18 years']
    for k in range(len(years)):
        p = prev.loc[(prev.DataValueType == 'Age-adjusted Prevalence') & (prev.YearStart == years[k]) & (prev.Stratification1 == ethnicity[r])]
        missing = (p['DataValue'].isnull().sum())
        if missing != 0:
            print(str(years[k]) + ": " + str(missing))

Missing values for the Question: Prevalence of diabetes - Age-adjusted Prevalence
Black, non-Hispanic:
2011: 14
2012: 15
2013: 14
2014: 15
2015: 14
2016: 15
2017: 15
2018: 14
Multiracial, non-Hispanic:
2011: 33
2012: 30
2013: 28
2014: 24
2015: 29
2016: 23
2017: 24
2018: 20
White, non-Hispanic:
2011: 3
2012: 3
2013: 3
2014: 3
2015: 3
2016: 2
2017: 3
2018: 3
Other, non-Hispanic:
2011: 11
2012: 12
2013: 10
2014: 7
2015: 8
2016: 9
2017: 8
2018: 8
Hispanic:
2011: 15
2012: 11
2013: 16
2014: 16
2015: 18
2016: 15
2017: 12
2018: 13


Next Question: **Glycosylated hemoglobin measurement among adults aged >= 18 years with diagnosed diabetes**

Lots of missing data. Missing data for overall, gender, and especially, ethnicity. There is too much missing data for the ethnicity, so, I will NOT be including this Question in my study/analysis.

In [23]:
g = gender_db.loc[gender_db.Question == 'Glycosylated hemoglobin measurement among adults aged >= 18 years with diagnosed diabetes']
r = race_db.loc[race_db.Question == 'Glycosylated hemoglobin measurement among adults aged >= 18 years with diagnosed diabetes']
print(g.Stratification1.unique())
print(r.Stratification1.unique())

['Male' 'Female']
['Black, non-Hispanic' 'Hispanic' 'Multiracial, non-Hispanic'
 'Other, non-Hispanic' 'White, non-Hispanic']


In [24]:
print("Missing values for the Question: Glycosylated hemoglobin - Age-adjusted Prevalence")
print("Overall:")
years = [2011,2012,2013,2014,2015,2016,2017,2018]
h1ac = overall_db.loc[overall_db.Question == 'Glycosylated hemoglobin measurement among adults aged >= 18 years with diagnosed diabetes']
for k in range(len(years)):
    h = h1ac.loc[(h1ac.DataValueType == 'Age-adjusted Prevalence') & (h1ac.YearStart == years[k])]
    missing = (h['DataValue'].isnull().sum())
    if missing != 0:
        print(str(years[k]) + ": " + str(missing))

Missing values for the Question: Glycosylated hemoglobin - Age-adjusted Prevalence
Overall:
2011: 12
2012: 19
2013: 14
2014: 29
2015: 15
2016: 38
2017: 14
2018: 34


In [25]:
print("Missing values for the Question: Glycosylated hemoglobin - Age-adjusted Prevalence")
years = [2011,2012,2013,2014,2015,2016,2017,2018]
gender = ['Male','Female']

for g in range(len(gender)):
    print(gender[g] + ":")
    h1ac = gender_db.loc[gender_db.Question == 'Glycosylated hemoglobin measurement among adults aged >= 18 years with diagnosed diabetes']
    for k in range(len(years)):
        h = h1ac.loc[(h1ac.DataValueType == 'Age-adjusted Prevalence') & (h1ac.YearStart == years[k]) & (h1ac.Stratification1 == gender[g])]
        missing = (h['DataValue'].isnull().sum())
        if missing != 0:
            print(str(years[k]) + ": " + str(missing))

Missing values for the Question: Glycosylated hemoglobin - Age-adjusted Prevalence
Male:
2011: 12
2012: 19
2013: 14
2014: 29
2015: 15
2016: 38
2017: 14
2018: 34
Female:
2011: 12
2012: 19
2013: 14
2014: 29
2015: 15
2016: 38
2017: 14
2018: 34


In [26]:
print("Missing values for the Question: Glycosylated hemoglobin - Age-adjusted Prevalence")
ethnicity = ['Black, non-Hispanic', 'Hispanic', 'Multiracial, non-Hispanic',
 'Other, non-Hispanic','White, non-Hispanic']

for r in range(len(ethnicity)):
    print(ethnicity[r] + ":")
    h1ac = race_db.loc[race_db.Question == 'Glycosylated hemoglobin measurement among adults aged >= 18 years with diagnosed diabetes']
    for k in range(len(years)):
        h = h1ac.loc[(h1ac.DataValueType == 'Age-adjusted Prevalence') & (h1ac.YearStart == years[k]) & (h1ac.Stratification1 == ethnicity[r])]
        missing = (h['DataValue'].isnull().sum())
        if missing != 0:
            print(str(years[k]) + ": " + str(missing))

Missing values for the Question: Glycosylated hemoglobin - Age-adjusted Prevalence
Black, non-Hispanic:
2011: 30
2012: 34
2013: 30
2014: 39
2015: 34
2016: 43
2017: 33
2018: 44
Hispanic:
2011: 44
2012: 47
2013: 44
2014: 47
2015: 40
2016: 51
2017: 41
2018: 50
Multiracial, non-Hispanic:
2011: 53
2012: 52
2013: 52
2014: 53
2015: 53
2016: 54
2017: 54
2018: 53
Other, non-Hispanic:
2011: 44
2012: 46
2013: 44
2014: 48
2015: 46
2016: 52
2017: 46
2018: 49
White, non-Hispanic:
2011: 14
2012: 21
2013: 16
2014: 30
2015: 16
2016: 40
2017: 15
2018: 35


Next Question: **Prevalence of high blood pressure among adults aged >= 18 years with diagnosed diabetes**

Data only available for the years years 2011,2013,2015,2017.
Not too much missing data for overall, gender. 
But: LOTS of missing data for ethnicity

In [27]:
g = gender_db.loc[gender_db.Question == 'Prevalence of high blood pressure among adults aged >= 18 years with diagnosed diabetes']
r = race_db.loc[race_db.Question == 'Prevalence of high blood pressure among adults aged >= 18 years with diagnosed diabetes']
print(g.Stratification1.unique())
print(r.Stratification1.unique())

['Female' 'Male']
['Multiracial, non-Hispanic' 'Black, non-Hispanic' 'White, non-Hispanic'
 'Hispanic' 'Other, non-Hispanic']


In [28]:
print("Missing values for the Question: Prevalence of High Blood Pressue - Age-adjusted Prevalence")
print("Overall:")
years = [2011,2013,2015,2017]
hbp = overall_db.loc[overall_db.Question == 'Prevalence of high blood pressure among adults aged >= 18 years with diagnosed diabetes']
for k in range(len(years)):
    h = hbp.loc[(hbp.DataValueType == 'Age-adjusted Prevalence') & (hbp.YearStart == years[k])]
    missing = (h['DataValue'].isnull().sum())
    if missing != 0:
        print(str(years[k]) + ": " + str(missing))

Missing values for the Question: Prevalence of High Blood Pressue - Age-adjusted Prevalence
Overall:
2011: 1
2013: 1
2015: 1
2017: 1


In [29]:
print("Missing values for the Question: Prevalence of High Blood Pressue - Age-adjusted Prevalence")
years = [2011,2013,2015,2017]
gender = ['Male','Female']

for g in range(len(gender)):
    print(gender[g] + ":")
    hbp = gender_db.loc[gender_db.Question == 'Prevalence of high blood pressure among adults aged >= 18 years with diagnosed diabetes']
    for k in range(len(years)):
        h = hbp.loc[(hbp.DataValueType == 'Age-adjusted Prevalence') & (hbp.YearStart == years[k]) & (hbp.Stratification1 == gender[g])]
        missing = (h['DataValue'].isnull().sum())
        if missing != 0:
            print(str(years[k]) + ": " + str(missing))

Missing values for the Question: Prevalence of High Blood Pressue - Age-adjusted Prevalence
Male:
2011: 1
2013: 1
2015: 1
2017: 1
Female:
2011: 1
2013: 1
2015: 1
2017: 1


In [30]:
print("Missing values for the Question: Prevalence of High Blood Pressue - Age-adjusted Prevalence")
years = [2011,2013,2015,2017]
ethnicity = ['Multiracial, non-Hispanic', 'Black, non-Hispanic', 'White, non-Hispanic',
 'Hispanic', 'Other, non-Hispanic']

for r in range(len(ethnicity)):
    print(ethnicity[r] + ":")
    hbp = race_db.loc[race_db.Question == 'Prevalence of high blood pressure among adults aged >= 18 years with diagnosed diabetes']
    for k in range(len(years)):
        h = hbp.loc[(hbp.DataValueType == 'Age-adjusted Prevalence') & (hbp.YearStart == years[k]) & (hbp.Stratification1 == ethnicity[r])]
        missing = (h['DataValue'].isnull().sum())
        if missing != 0:
            print(str(years[k]) + ": " + str(missing))     

Missing values for the Question: Prevalence of High Blood Pressue - Age-adjusted Prevalence
Multiracial, non-Hispanic:
2011: 48
2013: 51
2015: 51
2017: 52
Black, non-Hispanic:
2011: 23
2013: 23
2015: 22
2017: 25
White, non-Hispanic:
2011: 3
2013: 3
2015: 3
2017: 3
Hispanic:
2011: 36
2013: 34
2015: 36
2017: 33
Other, non-Hispanic:
2011: 39
2013: 33
2015: 39
2017: 37


Next Question: **Prevalence of high cholesterol among adults aged >= 18 years with diagnosed diabetes**

Just like high blood pressure, this variable has data available for overall and gender, but not ethnicity.

In [31]:
g = gender_db.loc[gender_db.Question == 'Prevalence of high cholesterol among adults aged >= 18 years with diagnosed diabetes']
r = race_db.loc[race_db.Question == 'Prevalence of high cholesterol among adults aged >= 18 years with diagnosed diabetes']
print(g.Stratification1.unique())
print(r.Stratification1.unique())

['Male' 'Female']
['Black, non-Hispanic' 'Hispanic' 'Multiracial, non-Hispanic'
 'Other, non-Hispanic' 'White, non-Hispanic']


In [32]:
print("Missing values for the Question: Prevalence of High Cholestrol - Age-adjusted Prevalence")
print("Overall:")
years = [2011,2013,2015,2017]
ch = overall_db.loc[overall_db.Question == 'Prevalence of high cholesterol among adults aged >= 18 years with diagnosed diabetes']
for k in range(len(years)):
    c = ch.loc[(ch.DataValueType == 'Age-adjusted Prevalence') & (ch.YearStart == years[k])]
    missing = (c['DataValue'].isnull().sum())
    if missing != 0:
        print(str(years[k]) + ": " + str(missing))

Missing values for the Question: Prevalence of High Cholestrol - Age-adjusted Prevalence
Overall:
2011: 1
2013: 1
2015: 1
2017: 1


In [33]:
print("Missing values for the Question: Prevalence of High Cholestrol - Age-adjusted Prevalence")
years = [2011,2013,2015,2017]
gender = ['Male','Female']

for g in range(len(gender)):
    print(gender[g] + ":")
    ch = gender_db.loc[gender_db.Question == 'Prevalence of high cholesterol among adults aged >= 18 years with diagnosed diabetes']
    for k in range(len(years)):
        c = ch.loc[(ch.DataValueType == 'Age-adjusted Prevalence') & (ch.YearStart == years[k]) & (ch.Stratification1 == gender[g])]
        missing = (c['DataValue'].isnull().sum())
        if missing != 0:
            print(str(years[k]) + ": " + str(missing))

Missing values for the Question: Prevalence of High Cholestrol - Age-adjusted Prevalence
Male:
2011: 1
2013: 1
2015: 1
2017: 1
Female:
2011: 1
2013: 1
2015: 1
2017: 1


In [34]:
print("Missing values for the Question: Prevalence of High Cholestrol - Age-adjusted Prevalence")
years = [2011,2013,2015,2017]
ethnicity = ['Black, non-Hispanic', 'Hispanic', 'Multiracial, non-Hispanic',
 'Other, non-Hispanic','White, non-Hispanic']

for r in range(len(ethnicity)):
    print(ethnicity[r] + ":")
    ch = race_db.loc[race_db.Question == 'Prevalence of high cholesterol among adults aged >= 18 years with diagnosed diabetes']
    for k in range(len(years)):
        c = ch.loc[(ch.DataValueType == 'Age-adjusted Prevalence') & (ch.YearStart == years[k]) & (ch.Stratification1 == ethnicity[r])]
        missing = (c['DataValue'].isnull().sum())
        if missing != 0:
            print(str(years[k]) + ": " + str(missing)) 

Missing values for the Question: Prevalence of High Cholestrol - Age-adjusted Prevalence
Black, non-Hispanic:
2011: 23
2013: 25
2015: 23
2017: 25
Hispanic:
2011: 36
2013: 37
2015: 36
2017: 33
Multiracial, non-Hispanic:
2011: 49
2013: 51
2015: 51
2017: 52
Other, non-Hispanic:
2011: 39
2013: 36
2015: 40
2017: 38
White, non-Hispanic:
2011: 3
2013: 3
2015: 3
2017: 3


So, I will be looking at the following questions:
- Mortality with diabetes reported as any listed cause of death
- Prevalence of diagnosed diabetes among adults aged >= 18 years
- Prevalence of high blood pressure among adults aged >= 18 years with diagnosed diabetes
- Prevalence of high cholesterol among adults aged >= 18 years with diagnosed diabetes

In [35]:
overall_db.to_csv(r'...\Overall_Diabetes_CDI.csv', index=False)
gender_db.to_csv(r'...\Gender_Diabetes_CDI.csv', index=False)
race_db.to_csv(r'...\Race_Diabetes_CDI.csv', index=False)